In [ ]:
import serial

In [ ]:
class RP7972A():
    """Esta clase pide y mide los datos de la fuente, para luego ser procesados"""
    def __init__(self, curr_initialFreq, curr_finalFreq, curr_amplitude, curr_sweepPoints):
        """Define variables de entrada como variables globales en la clase"""
        self.curr_initialFreq = curr_initialFreq
        self.curr_finalFreq = curr_finalFreq
        self.curr_amplitude = curr_amplitude
        self.curr_sweepPoints = curr_sweepPoints

    def generate_arb(self, ):
        """Para generar las señales de corriente que se envian a la fuente"""
        """INFO: service manual, cap 4, Programming an Arbitrary Waveform"""

    def measure_voltage():
        """Para configurar como tomar datos"""
        
    # TOOLBOX
    def sweep_frequencies(self):
        """A partir de las frecuencias inicial y final y la cantidad de sweep points se definen las frecuencias sobre las que se hará el barrido"""
        #TODO escribir funcion con curr_initialFreq, curr_finalFreq, curr_sweepPoints
    def cycles_per_frequency(self, sweep_freqs):
        """Switch para definir ciclos por frecuencia, recibe un array de frequencias por las que se hará el sweep, saca un diccionario"""


class HMI():
    """Esta clase es la interfaz entre el usuario y nuestro programa"""
    def __init__():


class EIS():
    """Una vez tengamos los datos, se procesaran con esta clase"""
    

In [ ]:
"""snippet for serial communication with the instrument"""
# validate_serial.py
import serial

COM_PORT = "COM5"  # Instrument port location
TIMEOUT = 1
CHECK_COMMAND = "*IDN?\n"  # Terminate with newline

with serial.Serial(
  COM_PORT,
  write_timeout=TIMEOUT,
  timeout=TIMEOUT) as serial_connection:
    serial_connection.write(CHECK_COMMAND.encode())  # Send command
    response = serial_connection.readline().decode()  # read response
print(response)